In [2]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None  # default='warn'


In [5]:
dfChild = pd.read_csv("Child.csv")
dfParent = pd.read_csv("Parent.csv")
dfOrder = pd.read_csv("Order.csv")

dfReqStockoutProductDetail = pd.read_csv("ReqStockoutProductDetail.csv") # Request Stockout Product Detail
dfOutSourcingDetail = pd.read_csv("OutSourcingDetail.csv") # Outsourcing Detail
dfReturnProduct = pd.read_csv("ReturnProduct.csv") # Return Product
dfPurchaseOrder = pd.read_csv("PurchaseOrder.csv") # Purchase Order
dfStockInPurchaseOrder = pd.read_csv("StockInPurchaseOrder.csv") # Purchase Order
dfOutSourcingQuota = pd.read_csv("OutSourcingQuota.csv")
# dfOutSourcing = pd.read_csv("OutSourcing.csv")



FileNotFoundError: [Errno 2] No such file or directory: 'Parent.csv'

In [ ]:
dfOutSourcingQuota

In [ ]:
dfOutSourcingQuota.merge(dfChild , left_on = 'product_id' , right_on= 'parent_product' , how = 'inner')

In [ ]:
dfChild

In [ ]:
dfChild[dfChild['child_product'] == 84]

In [ ]:
dfPurchaseOrder

In [ ]:
dfStockInPurchaseOrder

In [ ]:
dfPurchaseOrder = dfPurchaseOrder.merge(dfStockInPurchaseOrder,on = 'product_id' , how = 'outer')
dfPurchaseOrder = dfPurchaseOrder.replace(np.nan, 0)
dfPurchaseOrder["material_not_in_stock"] = dfPurchaseOrder["purchase_order_quantity"] - dfPurchaseOrder['quantity_in_stock_purchase_order']
dfPurchaseOrder[dfPurchaseOrder['material_not_in_stock'] < 0] = 0
dfPurchaseOrder

In [ ]:
dfReturnProduct

In [ ]:
dfParent

In [ ]:
dfParent.loc[dfParent['parent_product'] == 8422]

In [ ]:
dfOrder_drop_dup = dfOrder.drop_duplicates(subset=['order_code', 'product_code'], keep='first').reset_index(drop= True)
dfOrder_groupby_sum = dfOrder.groupby(['order_code', 'product_code'], as_index=False)['stockout_quantity'].sum()
dfOrder_groupby_sum.sort_values(by = ['stockout_quantity'])
dfOrder = pd.merge(dfOrder_drop_dup , dfOrder_groupby_sum, on=['order_code', 'product_code'], how='inner').drop(columns = ['stockout_quantity_x']).rename(columns = {"stockout_quantity_y":"stockout_quantity"})
dfOrder

In [ ]:
dfProductionOrder = dfOrder[['parent_product' , 'production_order_quantity', 'stockout_quantity']]
dfProductionOrder.sort_values(by = ['parent_product'])
dfProductionOrderDetail = dfProductionOrder.groupby(['parent_product'], as_index=False)['production_order_quantity' , 'stockout_quantity'].sum()
dfProductionOrderDetail

In [ ]:
dfOutSourcingDetail.head(10)

In [ ]:
df = dfReqStockoutProductDetail.merge(dfOutSourcingDetail, on = 'product_id' , how = 'outer')
df = df.merge(dfReturnProduct, on = 'product_id' , how = 'outer')
df.rename(columns = {'product_id':'parent_product'}, inplace = True)
df = dfProductionOrderDetail.merge(df, on = 'parent_product' , how = 'outer')
df = dfParent.merge(df, on = 'parent_product' , how = 'outer').drop(columns = ['is_outsourcing'])
df = df.fillna(0).reset_index(drop= True)
df

In [ ]:
df.loc[df['parent_product'] == 3802]

In [ ]:
df['total_production_need'] =  df['production_order_quantity'] + df['quantity_req_stockout'] + df['outsourcing_quantity'] + df['return_request_quantity']
df['total_exported'] = df['stockout_quantity'] + df['exported_req_quantity'] + df['quantity_stock_in'] + df['returned_quantity']

In [ ]:
df['temporary_production_inventory'] =  df['product_quantity'] - (df['total_production_need'] - df['total_exported'])
df

In [ ]:
dfProductionNeedsAndExported = df
dfProductionNeedsAndExported

In [ ]:
dfChild

In [ ]:
dfParent[dfParent['parent_product'] == 86]

In [ ]:
dfParent.loc[dfParent['parent_product'] == 8422]

In [ ]:
dfOrder

In [ ]:
dfOrder.loc[dfOrder['order_code'] == "NXK-BLACK-01295-VM-Gray-L-POJK3G"]

In [ ]:
dfOrder_drop_dup = dfOrder.drop_duplicates(subset=['order_code', 'product_code'], keep='first').reset_index(drop= True)
dfOrder_groupby_sum = dfOrder.groupby(['order_code', 'product_code'], as_index=False)['stockout_quantity'].sum()
dfOrder_groupby_sum.sort_values(by = ['stockout_quantity'])
dfOrder = pd.merge(dfOrder_drop_dup , dfOrder_groupby_sum, on=['order_code', 'product_code'], how='inner').drop(columns = ['stockout_quantity_x']).rename(columns = {"stockout_quantity_y":"stockout_quantity"})
dfOrder

In [ ]:
dfChild2 = dfChild.drop_duplicates(keep='first').reset_index(drop = True)
dfChild2 = dfChild2.sort_values(by=['parent_product']).reset_index(drop = True)
dfChild3 = pd.merge(dfChild2 , dfParent[['parent_product','product_quantity','temporary_quantity']], left_on='child_product', right_on='parent_product', how='inner').drop(columns = ['parent_product_y']).rename(columns = {"parent_product_x":"parent_product"})
dfChild3 = dfChild3.reset_index(drop = True)
g = dfChild3.groupby(["parent_product"]).cumcount().add(1)
dfChildMerge = dfChild3.set_index(["parent_product", g]).unstack(fill_value=0).sort_index(axis=1, level=1)
dfChildMerge.columns = ["{}{}".format(a, b) for a, b in dfChildMerge.columns]

dfChildMerge = dfChildMerge.reset_index()

dfChildMerge = dfChildMerge.replace(np.nan, 0)
dfChildMerge

In [ ]:
dfChildMerge[dfChildMerge['parent_product'] == 8522]

In [ ]:
dfChildMerge[dfChildMerge['quota5'] != 0]

In [ ]:
def mergeMaterialOrder(dfChild, dfParent, dfOrder):
   dfChild2 = dfChild.drop_duplicates(keep='first').reset_index(drop = True)
   dfChild2 = dfChild2.sort_values(by=['parent_product']).reset_index(drop = True)
   dfChild3 = pd.merge(dfChild2 , dfParent[['parent_product','product_quantity','temporary_quantity']], left_on='child_product', right_on='parent_product', how='inner').drop(columns = ['parent_product_y']).rename(columns = {"parent_product_x":"parent_product"})
   dfChild3 = dfChild3.reset_index(drop = True)
   g = dfChild3.groupby(["parent_product"]).cumcount().add(1)
   dfChildMerge = dfChild3.set_index(["parent_product", g]).unstack(fill_value=0).sort_index(axis=1, level=1)
   dfChildMerge.columns = ["{}{}".format(a, b) for a, b in dfChildMerge.columns]

   dfChildMerge = dfChildMerge.reset_index()

   dfChildMerge = dfChildMerge.replace(np.nan, 0)
   dfTotalMate = pd.merge(dfParent, dfChildMerge, on='parent_product', how='left')
   dfTotalMate = dfTotalMate.replace(np.nan, 0)
   
   dfTotalMate = dfTotalMate.drop(dfTotalMate[(dfTotalMate['child_product1'] == 0) & (dfTotalMate['is_outsourcing'] == 1)].index).reset_index(drop = True)
   new_col1 = ['parent_product','product_code' , 'product_quantity' ,'temporary_quantity', 'is_outsourcing', 
      'product_code1', 'child_product1', 'quota1',
      'product_quantity1', 'temporary_quantity1', 'product_code2',
      'child_product2', 'quota2', 'product_quantity2',
      'temporary_quantity2', 'product_code3', 'child_product3', 'quota3',
      'product_quantity3', 'temporary_quantity3', 'product_code4',
      'child_product4', 'quota4', 'product_quantity4',
      'temporary_quantity4', 'product_code5', 'child_product5', 'quota5',
      'product_quantity5', 'temporary_quantity5']
   
   dfTotalMate= dfTotalMate.reindex(columns=new_col1)
   dfTotalMate = dfTotalMate.sort_values(by= ['parent_product']).reset_index(drop = True) 

   dfTotal = pd.merge(dfOrder , dfTotalMate, left_on='product_code', right_on='product_code', how='left')
   dfTotal = dfTotal.reset_index(drop = True)
   new_col2 = ['order_code' , 'position' , 'total_quota', 'stockout_quantity',
      'parent_product' , 'product_code' , 'product_quantity',
      'temporary_quantity', 'is_outsourcing' , 'product_code1', 'child_product1', 'quota1',
      'product_quantity1', 'temporary_quantity1', 'product_code2',
      'child_product2', 'quota2', 'product_quantity2',
      'temporary_quantity2', 'product_code3', 'child_product3', 'quota3',
      'product_quantity3', 'temporary_quantity3', 'product_code4',
      'child_product4', 'quota4', 'product_quantity4',
      'temporary_quantity4', 'product_code5', 'child_product5', 'quota5',
      'product_quantity5', 'temporary_quantity5']
   
   dfTotal=dfTotal.reindex(columns=new_col2)

   dfTotal = dfTotal[:1000] 
   dfTotalReverse = dfTotal[::-1]
   dfTotalReverse = dfTotalReverse.reset_index(drop = True)
   dfTotalReverse = dfTotalReverse.drop(dfTotalReverse[np.isnan(dfTotalReverse['parent_product']) == True].index).reset_index(drop = True)
   return dfTotalMate, dfTotalReverse

In [ ]:
dfTotalMate, dfTotalReverse = mergeMaterialOrder(dfChild , dfParent, dfOrder)
dfTotalMate

In [ ]:
dfTotalReverse

In [ ]:
dict_evaluated = {"order_code":[] , "position":[], "product_code":[] ,  "comment" : [] ,"total_quota": [] , 
                  "stockout_quantity": [] , "temporary_quantity" : [] , "lack_of_material" : [] ,  "quota" : [] , 
                  "lack_of_quantity": [], "is_outsourcing" : []}
listColMateTtReverse = [9,14,19,24,29]
listColMate = [5,10,15,20,25] 

def updateDictEvaluated(dict_evaluated, aMate , comment, total_quota, stockout_quantity, 
                        temporary_quantity, lack_of_material, quota , lack_of_quantity , is_outsourcing):
    dict_evaluated["order_code"].append(aMate['order_code'])
    dict_evaluated["position"].append(aMate['position'])
    dict_evaluated["product_code"].append(aMate['product_code'])
    dict_evaluated["comment"].append(comment)
    dict_evaluated["total_quota"].append(total_quota)
    dict_evaluated["stockout_quantity"].append(stockout_quantity)
    dict_evaluated["temporary_quantity"].append(temporary_quantity)
    dict_evaluated["lack_of_material"].append(lack_of_material)
    dict_evaluated["quota"].append(quota)
    dict_evaluated["lack_of_quantity"].append(lack_of_quantity)
    dict_evaluated["is_outsourcing"].append(is_outsourcing)

In [ ]:
def calLevel1(ahat, total_missing_quota, dfParent):
    sub = ahat['temporary_quantity'] - total_missing_quota
    indexCode = dfParent[dfParent['product_code'] == ahat['product_code']].index[0] 
    dfParent.at[indexCode , 'temporary_quantity'] = sub
    sub = np.round(sub, 2)
    return sub, dfParent

In [ ]:
# update the quantity of material after using it
def replaceNewValue(aMate, dfParent , sub):
    update_sub = 0
    for i in listColMate: 
        if(aMate[i] != 0):
            indexCode = dfParent[dfParent['product_code'] == aMate[i]].index[0]
            update_sub = aMate[i+4] + (sub / aMate[i+2])
            dfParent.at[indexCode , 'temporary_quantity'] = round(update_sub,2)
        else:
            break
    return dfParent

In [ ]:
dfTotalReverse.loc[dfTotalReverse["product_code"] == "5-DAFELT-BLK-07-TA"]

In [ ]:
def calLevel2(ahat , product_id, dfTotalMate , sub1 , sub , dict_evaluated, dfParent, dfTotalReverse , quota):
    dict_notenough = {"product_id": [], "index_min": [] , "quota": [] ,"quantity_product": [] , "temp_quantity": []}
    count = negative = positive = 0
    aMate = dfTotalMate.loc[np.where(dfTotalMate['parent_product'] == product_id)[0][0]] 
    for i in listColMate: 
        if (aMate[5] == 0):
            return updateDictEvaluated(dict_evaluated, ahat, "not enough", ahat['total_quota'] ,ahat['stockout_quantity'] ,sub1, aMate[i-4], quota , sub , 0)
            
        elif(aMate[i] != 0):
            slSC = aMate[i+4] * aMate[i+2]
            if((slSC + sub1) < 0):
                dict_notenough['product_id'].append(aMate[i+1]) 
                dict_notenough['index_min'].append(i) 
                dict_notenough['quota'].append(quota * aMate[i+2]) 
                dict_notenough['quantity_product'].append(slSC) 
                dict_notenough['temp_quantity'].append(np.round((slSC + sub1)/aMate[i+2] , 2)) 
                negative = negative + 1
            else:
                positive = positive + 1
            count = count + 1
        else:
            break
                
    df_notenough = pd.DataFrame(dict_notenough)
        
    dfParent = replaceNewValue(aMate, dfParent , sub)
    dfTotalMate,dfTotalReverse = mergeMaterialOrder(dfChild , dfParent, dfOrder)
    
    if(count == positive):
        return updateDictEvaluated(dict_evaluated, ahat, "enough", ahat['total_quota'] , ahat['stockout_quantity'], sub1, np.NaN, quota , np.NaN, np.NaN)    
    else:    
        for i in range(0, negative):
            return calLevel2(ahat, df_notenough.loc[i]['product_id'], dfTotalMate , sub1, df_notenough.loc[i]['temp_quantity'], dict_evaluated, dfParent, dfTotalReverse, df_notenough.loc[i]['quota'])
        


In [ ]:
def primary(dfTotalReverse , dfParent, dfTotalMate, index):
    ahat = dfTotalReverse.loc[index]
    total_missing_quota = ahat['total_quota'] - ahat['stockout_quantity'] 
    if(total_missing_quota <= 0.0):
        updateDictEvaluated(dict_evaluated, ahat, "exported", ahat['total_quota'] , ahat['stockout_quantity'], np.NAN, np.NAN , np.NAN , np.NAN, np.NAN)
    else: 
        if(ahat['is_outsourcing'] == 0):
            sub, dfParent = calLevel1(ahat, total_missing_quota, dfParent)
            dfTotalMate,dfTotalReverse = mergeMaterialOrder(dfChild , dfParent, dfOrder)
            if(sub > 0):
                updateDictEvaluated(dict_evaluated, ahat, "enough", ahat['total_quota'] , ahat['stockout_quantity'], sub, np.NAN, np.NAN, np.NAN , 0)
            else:
                updateDictEvaluated(dict_evaluated, ahat, "not enough", ahat['total_quota'] , ahat['stockout_quantity'], sub , ahat["product_code"],np.NAN, sub , 0)
        else:
            sub1, dfParent = calLevel1(ahat, total_missing_quota, dfParent)
            sub = np.NAN
            dfTotalMate,dfTotalReverse = mergeMaterialOrder(dfChild , dfParent, dfOrder)
            if(sub1 > 0):
                updateDictEvaluated(dict_evaluated, ahat, "enough", ahat['total_quota'] , ahat['stockout_quantity'], sub1 , np.NAN,  np.NAN, np.NAN , 1)
            else:
                calLevel2(ahat , ahat['parent_product'], dfTotalMate , sub1 , sub , dict_evaluated, dfParent, dfTotalReverse , 1)
    return dfTotalMate, dfTotalReverse
            

In [ ]:
dfTotalReverse

In [ ]:
for i in range(0, len(dfTotalReverse)):
    dfTotalMate, dfTotalReverse = primary(dfTotalReverse ,dfParent, dfTotalMate, i)

In [ ]:
dfEvaluted = pd.DataFrame.from_dict(dict_evaluated)
# dfEvaluted = dfEvaluted[::-1].reset_index(drop = True)
dfEvaluted.to_excel('data.xlsx')
dfEvaluted

In [ ]:
listOrder = dfEvaluted['order_code'].unique()
newDictEvalue = {"order_code": [] , "comment": []}
for i in listOrder:
    listEva = dfEvaluted.loc[dfEvaluted['order_code'] == i]['comment'].unique()
    if (len(listEva) == 1):
        newDictEvalue["comment"].append(listEva[0])
        newDictEvalue["order_code"].append(i)
    else:
        if(listEva[0] != listEva[1]):
            newDictEvalue["comment"].append("not enough")
            newDictEvalue["order_code"].append(i)
        else:
            newDictEvalue["comment"].append(listEva)
        
dfEvalutedTt = pd.DataFrame.from_dict(newDictEvalue)
# dfEvalutedTt

In [ ]:
# dfEvalutedTt.loc[dfEvalutedTt["quantity"] == "enough"]

In [ ]:
# dfEvalutedTt.loc[dfEvalutedTt["quantity"] == "not enough"]